PREREQUISTES

VAULT SECRET OCID and ADB OCID need to be completed in ora23ai_connection before execution of the notebook.

In [ ]:
!python3 -m pip install oracledb langchain_community oci gradio pypdf langchain_cohere==0.1.9 langchain-huggingface langchain_experimental openai langchain_openai langchain_nvidia_ai_endpoints

In [ ]:
from ora23ai_gradio_chatbot import setup_chatbot, clean_up_vector_db
from langchain_cohere import CohereEmbeddings
from langchain_cohere.chat_models import ChatCohere
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_openai import ChatOpenAI
from openai import OpenAI

embedding_models = {
    "nv-embedqa-e5-v5": {
        "model": "nvidia/nv-embedqa-e5-v5",
        "embedding_function": lambda api_key: NVIDIAEmbeddings(
            base_url = "http://nemo-embedding-ms:8080/v1",
            model="nvidia/nv-embedqa-e5-v5"
        ),
        "size": 1024
    },
    "cohere": {
        "model": "embed-english-light-v3.0",
        "embedding_size": 384,
        "embedding_function": lambda api_key: CohereEmbeddings(
            cohere_api_key=api_key,
            model="embed-english-light-v3.0"
        ),
        "size": 384
    }
}

llm_models = {
    "gemma-2b": {
        "model": "google/gemma-2b",
        "llm_function": lambda api_key, kwargs={}: ChatOpenAI(
            base_url="https://integrate.api.nvidia.com/v1",
            model="google/gemma-2b",
            api_key=api_key,
            **kwargs
        )
    },
    "cohere-chat": {
        "model": "command-r-plus",
        "llm_function": lambda api_key, kwargs={}: ChatCohere(
            cohere_api_key=api_key,
            **kwargs
        )
    }
}

try:
    model = ""
    client = OpenAI(
      base_url = "http://llm/v1",
      api_key = "dummy"
    )
    available_models = client.models.list()
    if len(available_models.data):
        model = available_models.data[0].id
        print(f"Discovered model is: {model}")
    else:
        print("No model discovered")
except Exception as e:
    print(e)

if model:
    llm_models[f'{model}']= {
        "model": model,
        "llm_function": lambda api_key, kwargs={}: ChatOpenAI(
            base_url="http://llm/v1",
            model=model,
            api_key="dummy",
            **kwargs
        )
    }


clean_up_vector_db()

demo = setup_chatbot(llm_models, embedding_models)
demo.queue().launch(share=True)